In [1]:
import gpn.model
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel, AutoModelForMaskedLM, AutoTokenizer
import h5py
import sys
from tqdm import tqdm
import numpy as np
sys.path.append('../')
import utils
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000").to('cuda')
# tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000")
# model = AutoModel.from_pretrained('songlab/gpn-brassicales').to('cuda')
# tokenizer = AutoTokenizer.from_pretrained('songlab/gpn-brassicales')
 
model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000")
model.eval();




celltype = 'HepG2'

file = h5py.File('/home/ztang/multitask_RNA/data/mtsplice/delta_logit.h5','r')

/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000 were not used when initializing ConvNetModel: ['cls.decoder.3.bias', 'cls.decoder.0.weight', 'cls.decoder.0.bias', 'cls.decoder.2.weight', 'cls.decoder.2.bias', 'cls.decoder.3.weight']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [2]:
gpn_output = h5py.File('../../data/mtsplice/gpn_mt.h5','w')
batch_size = 32
for label in ('valid','test','train'):
    l_cache = []
    r_cache = [] 
    for i in tqdm(range(0,len(file['x_'+label]),batch_size)):
        seq = file['x_'+label][i:i+batch_size].astype('int')
        seq = utils.onehot_to_seq(seq)
        clean_seq = [s if 'N' not in s else s.replace('N','[PAD]') for s in seq ]
        input_ids = tokenizer(clean_seq, return_tensors="pt", return_attention_mask=False, return_token_type_ids=False)["input_ids"]
        l_input = input_ids[:,:400]
        r_input = input_ids[:,400:]
        with torch.no_grad():
            l_output = model(l_input.to('cuda')).last_hidden_state.cpu().detach().numpy()
            r_output = model(r_input.to('cuda')).last_hidden_state.cpu().detach().numpy()
        l_cache.extend(l_output)
        r_cache.extend(r_output)
    gpn_output.create_dataset(name='xl_'+label,data = np.array(l_cache),dtype = 'float32')
    gpn_output.create_dataset(name='xr_'+label,data = np.array(r_cache),dtype = 'float32')
    gpn_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='float32') 
gpn_output.close()

  0%|          | 0/34 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 1189/1189 [02:12<00:00,  8.95it/s]
